In [1]:
import logging
import warnings

from pyspark.sql import SparkSession

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)
warnings.filterwarnings("ignore")

spark = SparkSession.builder \
    .master("local[4]") \
    .config("spark.driver.memory", "8g") \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

22/11/12 18:37:19 WARN Utils: Your hostname, MacBook-Pro-4.local resolves to a loopback address: 127.0.0.1; using 192.168.178.19 instead (on interface en0)
22/11/12 18:37:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/12 18:37:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from classify_amazonreviews_sparkml import run_pipeline

start_date = '2014-01-01'
split_date = '2014-12-01'

run_pipeline(spark, start_date, split_date)

Accuracy: 0.8175150408977219


In [3]:
from freamon.adapters.pyspark.provenance import from_trace

view_generator = from_trace()

INFO:root:Registering source 0 with columns: ['_c0', 'marketplace', 'customer_id', 'review_id', 'product_id', 'vine', 'third_party', 'review_headline', 'review_body', 'review_date', 'prov_id_source_0']
INFO:root:Registering source 1 with columns: ['_c0', 'product_id', 'product_parent', 'product_title', 'category_id', 'prov_id_source_1']
INFO:root:Registering source 2 with columns: ['_c0', 'id', 'category', 'prov_id_source_2']
INFO:root:Registering source 3 with columns: ['_c0', 'review_id', 'star_rating', 'helpful_votes', 'total_votes', 'prov_id_source_3']
INFO:root:Computing train features and predictions...
INFO:root:Computing test features and predictions...                            
INFO:root:Generating virtual train view via                                     


            CREATE OR REPLACE VIEW _freamon_virtual_train_view AS 
              SELECT * EXCLUDE (prov_id_source_0, prov_id_source_1, prov_id_source_2, prov_id_source_3)
              FROM _freamon_train t
            

In [4]:
materialized_view = view_generator.test_view(
    sliceable_by=['category', 'star_rating'], 
    with_features=False, 
    with_y_true=True, 
    with_y_pred=True)

materialized_view

,category,star_rating,y_true,y_pred
0,Digital_Software,5,0,0.0
1,Digital_Software,4,0,0.0
2,Digital_Software,5,0,0.0
3,Digital_Video_Games,5,0,0.0
4,Digital_Software,5,0,0.0
...,...,...,...,...
29581,Digital_Video_Games,3,0,0.0
29582,Digital_Video_Games,5,1,0.0
29583,Digital_Software,3,1,1.0
29584,Digital_Video_Games,3,0,0.0


In [5]:
from fairlearn.metrics import MetricFrame, false_positive_rate
from sklearn.metrics import recall_score

materialized_view['rating'] = '(low rated)'
materialized_view['rating'].loc[materialized_view.star_rating.astype(int) > 3] = '(highly rated)'

fairness_metrics = MetricFrame(
    metrics={ 'tpr' : recall_score, 'fpr' : false_positive_rate },
    y_true=materialized_view.y_true,
    y_pred=materialized_view.y_pred,
    sensitive_features=materialized_view.category + ' ' + materialized_view.rating
)

fairness_metrics.by_group

,tpr,fpr
sensitive_feature_0,,
Digital_Software (highly rated),0.259978,0.009565
Digital_Software (low rated),0.436495,0.030914
Digital_Video_Games (highly rated),0.274116,0.010854
Digital_Video_Games (low rated),0.514019,0.091127


In [6]:
view_generator.execute_query(
"""
SELECT 
    star_rating > 3 as top_rated,
    category = 'Digital_Video_Games' as digi_games,
    AVG(-(y_true * log(y_pred_proba) + (1 - y_true) * log(1.0 - y_pred_proba))) AS avg_loss,
    VARIANCE(-(y_true * log(y_pred_proba) + (1 - y_true) * log(1.0 - y_pred_proba))) AS var_loss,    
    COUNT(*) as size
    
FROM (
    SELECT star_rating, category, y_true, IF(y_pred=0, 0.00001, 0.99999) AS y_pred_proba 
    FROM _freamon_virtual_test_view
    )
GROUP BY GROUPING SETS ((star_rating > 3, category = 'Digital_Video_Games'), (star_rating > 3), 
    (category = 'Digital_Video_Games'))
"""
)

,top_rated,digi_games,avg_loss,var_loss,size
0,False,False,1.636109,5.504498,6708
1,True,False,0.778447,3.286601,9018
2,True,True,0.491115,2.214578,10181
3,False,True,1.087255,4.255292,3679
4,False,NaN,1.441709,5.130499,10387
5,True,NaN,0.626078,2.738540,19199
6,NaN,False,1.144287,4.412305,15726
7,NaN,True,0.649354,2.825296,13860
